In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
import pickle
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.sampler import Sampler
import torchvision.transforms as transforms
import torchvision.datasets as dset
from scipy.misc import imread
from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.rpn.bbox_transform import clip_boxes
from model.nms.nms_wrapper import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections
from model.utils.blob import im_list_to_blob
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet
from model.utils.net_utils import weights_normal_init, save_net, load_net, \
      adjust_learning_rate, save_checkpoint, clip_gradient
    

In [3]:
def _get_image_blob(im):
    """Converts an image into a network input.
    Arguments:
    im (ndarray): a color image in BGR order
    Returns:
    blob (ndarray): a data blob holding an image pyramid
    im_scale_factors (list): list of image scales (relative to im) used
      in the image pyramid
    """
    im_orig = im.astype(np.float32, copy=True)
    im_orig -= cfg.PIXEL_MEANS

    im_shape = im_orig.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])

    processed_ims = []
    im_scale_factors = []

    for target_size in cfg.TEST.SCALES:
        im_scale = float(target_size) / float(im_size_min)
        # Prevent the biggest axis from being more than MAX_SIZE
        if np.round(im_scale * im_size_max) > cfg.TEST.MAX_SIZE:
            im_scale = float(cfg.TEST.MAX_SIZE) / float(im_size_max)
        im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale,interpolation=cv2.INTER_LINEAR)
        im_scale_factors.append(im_scale)
        processed_ims.append(im)

    # Create a blob to hold the input images
    blob = im_list_to_blob(processed_ims)

    return blob, np.array(im_scale_factors)

In [4]:
cfg_from_file("cfgs/vgg16.yml")
set_cfgs = ['ANCHOR_SCALES', '[4, 8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]']
cfg_from_list(set_cfgs)
np.random.seed(cfg.RNG_SEED)
load_model_path = os.path.join("models/res101/holly/good_models/faster_rcnn_1_1_117000.pth")
holly_classes = np.asarray(['__background__', 'head'])
isCuda = True
cfg.CUDA = isCuda

In [5]:
def getDiff(old_tensor, new_tensor, seq, detail=False):
    total_difference = 0
    diff = 0
    original = 0  
    if "Conv2d" in str(old_tensor) and not hasattr(old_tensor, "__getitem__") :
        old_shape = old_tensor.weight.shape[0]
        new_shape = new_tensor.weight.shape[0]
        diff = old_shape - new_shape
        original += old_shape
        total_difference += diff
        if detail:
            print(str(e_old) + "Difference = {}".format(diff))
    elif "Sequential" in str(old_tensor) and hasattr(old_tensor, "__getitem__"):
        for j,b in enumerate(old_tensor):
            bn_old = vars(b)
            bn_new = vars(new_tensor[j])
            convs_old = bn_old["_modules"]
            convs_new = bn_new["_modules"]
            for k,v in convs_old.items():
                if "conv" in k:
                    old_shape = v.weight.shape[0]
                    original += old_shape
                    new_shape = convs_new[k].weight.shape[0]
                    diff = old_shape - new_shape
                    if detail:
                        print("S {}/ B {}/ {} Difference = {}".format(seq, j, k, diff))
                    total_difference += diff
    return original, total_difference

In [6]:
def make_cuda(isCuda, tensor):
    if isCuda:
        return tensor.cuda()
    return tensor

In [7]:
def getNumpy(tensor):
    return tensor.data.cpu().numpy()

def sortFilters(filters):
    #Sort filters by L1 Norm
    c = filters.reshape(-1, filters.shape[0])
    c = np.linalg.norm(c, ord=1, axis=0)
    d = np.zeros([2, c.shape[0]])
    i = np.argsort(c)
    d[0,:] = i
    d[1,:] = c[i]
    return d
def removeFilters(filters, percent, threshold=None):
    #Remove filters from numpy 
    t = threshold
    if threshold is None:
        t = np.median(filters[1,:]) - np.std(filters[1,:])
    mask = filters[1, :] > t
    numRemoved_1 = mask.sum()
    numRemoved_2 = int(numRemoved_1 * ((100. - percent) / 100.))
    numOfZeros = numRemoved_1 - numRemoved_2
    newFilters = filters[:,numRemoved_2:].copy()
    newFilters[1, 0:numOfZeros] =0
    return newFilters

def removePercent(filters, KeepPercent, PrunePercent):
    #Remove filters from numpy 
    numOfRemove = filters.shape[1] * PrunePercent / 100.
    numRemoved_1 = int(numOfRemove)
    numRemoved_2 = int(numRemoved_1 * ((100. - KeepPercent) / 100.))
    numOfZeros = numRemoved_1 - numRemoved_2
    newFilters = filters[:,numRemoved_2:].copy()
    print("NumOfZeros:")
    print(numOfZeros)
    newFilters[1, 0:numOfZeros] =0
    return newFilters

def sortRecoverFilters(l1Array, filters):
    #Recover the order of the pruned filters
    b = np.argsort(l1Array[0, :])
    c = l1Array[:,b]
    c = c[0,:].astype(int)
    shape = np.asarray(filters.shape)
    shape[0] = c.shape[0]
    newFilters = np.zeros(shape)
    newFilters = filters[c,:,:,:]
    return newFilters
def sortRecoverBatch(bn_tensor, index):
    #Batch pruning 
    index.sort()
    bn_rmean = bn_tensor.running_mean.cpu().numpy()
    bn_tensor.running_mean = torch.from_numpy(bn_rmean[index]).float().cuda()
    bn_rvar = bn_tensor.running_var.cpu().numpy()
    bn_tensor.running_var = torch.from_numpy(bn_rvar[index]).float().cuda()
    bn_weight = bn_tensor.weight.data.cpu().numpy()
    bn_tensor.weight.data = torch.from_numpy(bn_weight[index]).float().cuda()
    bn_bias = bn_tensor.bias.data.cpu().numpy()
    bn_tensor.bias.data = torch.from_numpy(bn_bias[index]).float().cuda()

def pruneConvLayers(tensor, percent = 10, threshold = None):
    #Prune out channels of convolutional layers
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = removeFilters(d, percent, threshold)
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvPercent(tensor, KeepPercent = 10, PrunePercent=10):
    #Prune out channels of convolutional layers
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = removePercent(d, KeepPercent, PrunePercent)
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvLowest(tensor, numKeep):
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = d[:,0:numKeep]
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvWithIndexes(tensor, rindexes):
    #Prune out channels of convolutional layers using indexes
    filters = getNumpy(tensor.weight)
    tensor.weight.data = torch.from_numpy(filters[rindexes]).float().cuda()
    tensor.out_channels = tensor.weight.data.shape[0]
    return tensor, rindexes
def pruneNextLayer(nextLayerTensor, prevOutput, rindexes=None):
    #Prune input channels of everything
    if "BatchNorm" in str(nextLayerTensor):
        nextLayerTensor.num_features = prevOutput
        sortRecoverBatch(nextLayerTensor, rindexes)
    elif "Conv2d" in str(nextLayerTensor):
        if nextLayerTensor.weight.shape[1] == prevOutput:
            return
        nextLayerTensor.in_channels = prevOutput
        nextConvWeight = getNumpy(nextLayerTensor.weight)
        if not rindexes is None:
            c = nextConvWeight[:,rindexes,:,:]
        else:
            c = nextConvWeight
        nextLayerTensor.weight.data = torch.from_numpy(c).float().cuda()
    elif "Linear" in str(nextLayerTensor):
        n = getNumpy(nextLayerTensor.weight)
        fc = n[:,rindexes]
        nextLayerTensor.in_features = fc.shape[1]
        nextLayerTensor.weight.data = torch.from_numpy(fc).float().cuda()
    return nextLayerTensor
def prune_bottleneck(bottlenecks, prevOutput, rindexesX, keeplast=False):
    for i,bottleneck in enumerate(bottlenecks):
        #Conv1 Prune
        currentTensor = pruneNextLayer(bottleneck.conv1, prevOutput, rindexesX) #Conv1 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv1) #Conv1 Output channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn1, currentTensor.out_channels, rindexes) #BN1 prune
        
        #Conv2 Prune
        currentTensor           = pruneNextLayer(bottleneck.conv2, currentTensor.num_features, rindexes) #Conv2 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv2) #Conv2 Input channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn2, currentTensor.out_channels, rindexes) #BN2 prune
        #Shortcut + conv2
        currentTensor           = pruneNextLayer(bottleneck.conv3, currentTensor.num_features, rindexes) #Conv3 input channels
        
        #Prune shortcut first
        if not bottleneck.downsample is None:
            b_neck = bottleneck.downsample
            pruneNextLayer(b_neck[0], prevOutput, rindexesX) #Shortcut Input Channels
            shorcutconv, rindexes = pruneConvLayers(b_neck[0]) #Shortcut output channels
            pruneNextLayer(b_neck[1], b_neck[0].out_channels, rindexes) #bn1 of shorcut
        
            if not keeplast:
                lastconv, rindexes = pruneConvWithIndexes(bottleneck.conv3, rindexes) #Conv3 output channels
            else:
                lastconv = bottleneck.conv3
        else:
            if not keeplast:
                lastconv, rindexes = pruneConvLowest(bottleneck.conv3, len(rindexesX))
            else:
                lastconv = bottleneck.conv3
            
        lastbn = pruneNextLayer(bottleneck.bn3, currentTensor.out_channels, rindexes) #BN3 prune
        num_output = lastconv.out_channels
        rindexesX = rindexes
    return bottleneck, num_output, rindexes

def prune_bottleneck_percent(bottlenecks, rindexesX, percents):
    for i,bottleneck in enumerate(bottlenecks):
        #Conv1 Prune
        currentTensor = pruneNextLayer(bottleneck.conv1, len(rindexesX), rindexesX) #Conv1 Input channels Prune
        currentTensor, rindexes = pruneConvPercent(bottleneck.conv1, percents[0]) #Conv1 Output channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn1, currentTensor.out_channels, rindexes) #BN1 prune
        
        #Conv2 Prune
        currentTensor           = pruneNextLayer(bottleneck.conv2, currentTensor.num_features, rindexes) #Conv2 Input channels Prune
        currentTensor, rindexes = pruneConvPercent(bottleneck.conv2, percents[1]) #Conv2 Input channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn2, currentTensor.out_channels, rindexes) #BN2 prune
        #Shortcut + conv2
        currentTensor           = pruneNextLayer(bottleneck.conv3, currentTensor.num_features, rindexes) #Conv3 input channels
        
        #Prune shortcut first
        if not bottleneck.downsample is None:
            b_neck = bottleneck.downsample
            pruneNextLayer(b_neck[0], len(rindexesX), rindexesX) #Shortcut Input Channels
            shorcutconv, rindexes = pruneConvPercent(b_neck[0], percents[2]) #Shortcut output channels
            pruneNextLayer(b_neck[1], b_neck[0].out_channels, rindexes) #bn1 of shorcut
        
            lastconv, rindexes = pruneConvWithIndexes(bottleneck.conv3, rindexes) #Conv3 output channels
        else:
            lastconv, rindexes = pruneConvLowest(bottleneck.conv3, len(rindexesX))
            
        lastbn = pruneNextLayer(bottleneck.bn3, currentTensor.out_channels, rindexes) #BN3 prune
        num_output = lastconv.out_channels
        rindexesX = rindexes
    return bottleneck, num_output, rindexes

def pruneCBias(convTensor, rindexes):
    convT = convTensor
    bias_data = convT.bias.data.cpu().numpy()
    convT.bias.data = torch.from_numpy(bias_data[rindexes]).float().cuda()
    return convT, rindexes


In [8]:
def pruneEverything():
    percent = 10
    prevIndex = 0
    prevOutput = 3
    rindexes = None
    for i,e in enumerate(fasterRCNN.RCNN_base):
        currentTensor = e
        if "Conv2d" in str(currentTensor) and not hasattr(currentTensor, "__getitem__"):
            if prevOutput != currentTensor.in_channels:
                currentTensor = pruneNextLayer(currentTensor, prevOutput)
            currentTensor, rindexes = pruneConvLayers(currentTensor, percent)
            prevOutput = currentTensor.out_channels

        elif "BatchNorm" in str(currentTensor) and not hasattr(currentTensor, "__getitem__"):
            if prevOutput != currentTensor.num_features:
                currentTensor = pruneNextLayer(currentTensor, prevOutput, rindexes)           
            prevOutput = currentTensor.num_features
        elif hasattr(currentTensor, "__getitem__"):
            b, prevOutput, rindexes = prune_bottleneck(currentTensor, prevOutput, rindexes)

        prevIndex = i
        prevTensor = currentTensor

    #Prune RoI pooling and FC classifiers
    b = pruneNextLayer(fasterRCNN.RCNN_rpn.RPN_Conv, prevOutput, rindexes)
    b, p, r = prune_bottleneck(fasterRCNN.RCNN_top[0], prevOutput, rindexes)
    fc = pruneNextLayer(fasterRCNN.RCNN_bbox_pred, p, r)
    cls_score = pruneNextLayer(fasterRCNN.RCNN_cls_score, p, r)

In [9]:
load_model_path = "pth_train_dir/vgg16/finished_models/vgg16_test_2_60000.pth"
fasterRCNN = vgg16(holly_classes, pretrained=False, class_agnostic=False)
fasterRCNN.create_architecture()
checkpoint = torch.load(load_model_path)
fasterRCNN.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']

In [10]:
def reload_model(weightsPTH):
    fasterRCNN = vgg16(holly_classes, pretrained=False, class_agnostic=False)
    fasterRCNN.create_architecture()
    checkpoint = torch.load(weightsPTH)
    fasterRCNN.load_state_dict(checkpoint['model'])
    fasterRCNN.cuda()
    if 'pooling_mode' in checkpoint.keys():
        cfg.POOLING_MODE = checkpoint['pooling_mode']
    return fasterRCNN

In [11]:
if isCuda:
    fasterRCNN.cuda()

In [12]:
def evalNetwork(modelFR, out=False):
    modelFR.eval()
    
    cfg.TRAIN.USE_FLIPPED = False
    imdb_name = "holly_test"
    imdb, roidb, ratio_list, ratio_index = combined_roidb(imdb_name, False)
    imdb.competition_mode(on=True)
    
    save_name = 'faster_rcnn_10'
    num_images = len(imdb.image_index)
    all_boxes = [[[] for _ in range(num_images)]
               for _ in range(imdb.num_classes)]

    output_dir = get_output_dir(imdb, save_name)

    dataset = roibatchLoader(roidb, ratio_list, ratio_index, 1, \
                            imdb.num_classes, training=False, normalize = False)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                shuffle=False, num_workers=0,
                                pin_memory=True)
    output_dir = "/tmp/outTest"
    vis = False
    thresh = 0.0
    
    data_iter = iter(dataloader)
    num_images = len(imdb.image_index)

    start = time.time()
    max_per_image = 100

    im_dataC = torch.FloatTensor(1)
    im_infoC = torch.FloatTensor(1)
    num_boxesC = torch.LongTensor(1)
    gt_boxesC = torch.FloatTensor(1)

    # ship to cuda
    im_dataC = make_cuda(isCuda, im_dataC)
    im_infoC = make_cuda(isCuda, im_infoC)
    num_boxesC = make_cuda(isCuda, num_boxesC)
    gt_boxesC = make_cuda(isCuda, gt_boxesC)

      # make variable
    im_dataC = Variable(im_dataC, volatile=True)
    im_infoC = Variable(im_infoC, volatile=True)
    num_boxesC = Variable(num_boxesC, volatile=True)
    gt_boxesC = Variable(gt_boxesC, volatile=True)

    _t = {'im_detect': time.time(), 'misc': time.time()}
    det_file = os.path.join(output_dir, 'detections.pkl')

    empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))
    for i in range(num_images):

        data = next(data_iter)
        im_dataC.data.resize_(data[0].size()).copy_(data[0])
        im_infoC.data.resize_(data[1].size()).copy_(data[1])
        gt_boxesC.data.resize_(data[2].size()).copy_(data[2])
        num_boxesC.data.resize_(data[3].size()).copy_(data[3])

        det_tic = time.time()
        rois, cls_prob, bbox_pred, \
        rpn_loss_cls, rpn_loss_box, \
        RCNN_loss_cls, RCNN_loss_bbox, \
        rois_label = modelFR(im_dataC, im_infoC, gt_boxesC, num_boxesC)

        scores = cls_prob.data
        boxes = rois.data[:, :, 1:5]

        if cfg.TEST.BBOX_REG:
              # Apply bounding-box regression deltas
            box_deltas = bbox_pred.data
            if cfg.TRAIN.BBOX_NORMALIZE_TARGETS_PRECOMPUTED:
              # Optionally normalize targets by a precomputed mean and stdev
                if False:#args.class_agnostic:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4)
                else:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4 * len(imdb.classes))

            pred_boxes = bbox_transform_inv(boxes, box_deltas, 1)
            pred_boxes = clip_boxes(pred_boxes, im_infoC.data, 1)
        else:
              # Simply repeat the boxes, once for each class
            pred_boxes = np.tile(boxes, (1, scores.shape[1]))

        pred_boxes /= data[1][0][2]

        scores = scores.squeeze()
        pred_boxes = pred_boxes.squeeze()
        det_toc = time.time()
        detect_time = det_toc - det_tic
        misc_tic = time.time()
        if vis:
            im = cv2.imread(imdb.image_path_at(i))
            im2show = np.copy(im)
        for j in range(1, imdb.num_classes):
            inds = torch.nonzero(scores[:,j]>thresh).view(-1)
              # if there is det
            if inds.numel() > 0:
                cls_scores = scores[:,j][inds]
                _, order = torch.sort(cls_scores, 0, True)
                if False:#args.class_agnostic:
                    cls_boxes = pred_boxes[inds, :]
                else:
                    cls_boxes = pred_boxes[inds][:, j * 4:(j + 1) * 4]

                cls_dets = torch.cat((cls_boxes, cls_scores.unsqueeze(1)), 1)
                # cls_dets = torch.cat((cls_boxes, cls_scores), 1)
                cls_dets = cls_dets[order]
                keep = nms(cls_dets, cfg.TEST.NMS)
                cls_dets = cls_dets[keep.view(-1).long()]
                if vis:
                    im2show = vis_detections(im2show, imdb.classes[j], cls_dets.cpu().numpy(), 0.3)
                all_boxes[j][i] = cls_dets.cpu().numpy()
            else:
                all_boxes[j][i] = empty_array

          # Limit to max_per_image detections *over all classes*
        if max_per_image > 0:
            image_scores = np.hstack([all_boxes[j][i][:, -1]
                                        for j in range(1, imdb.num_classes)])
            if len(image_scores) > max_per_image:
                image_thresh = np.sort(image_scores)[-max_per_image]
                for j in range(1, imdb.num_classes):
                    keep = np.where(all_boxes[j][i][:, -1] >= image_thresh)[0]
                    all_boxes[j][i] = all_boxes[j][i][keep, :]

        misc_toc = time.time()
        nms_time = misc_toc - misc_tic

        sys.stdout.write('im_detect: {:d}/{:d} {:.3f}s {:.3f}s   \r' \
          .format(i + 1, num_images, detect_time, nms_time))
        sys.stdout.flush()

        if vis:
            cv2.imwrite('result.png', im2show)
            pdb.set_trace()

    with open(det_file, 'wb+') as f:
        pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

    print('Evaluating detections')
    ap = imdb.evaluate_detections(all_boxes, output_dir, out)
    return ap

In [13]:
def getFeaturesMap(self, input, output):
    gpfm = F.avg_pool2d(Variable(output.data.cpu()), kernel_size=(output.data.size()[2], output.data.size()[3])).data.numpy().reshape((1,self.out_channels))
    if len(allFM[self]) == 0:
        allFM[self] = gpfm
    else:
        allFM[self] = np.concatenate((allFM[self], gpfm), axis=0)

In [14]:
def run1Forward(model, imagePath):
    model.eval()
    im_dataC = torch.FloatTensor(1)
    im_infoC = torch.FloatTensor(1)
    num_boxesC = torch.LongTensor(1)
    gt_boxesC = torch.FloatTensor(1)

    # ship to cuda
    im_dataC = make_cuda(isCuda, im_dataC)
    im_infoC = make_cuda(isCuda, im_infoC)
    num_boxesC = make_cuda(isCuda, num_boxesC)
    gt_boxesC = make_cuda(isCuda, gt_boxesC)

      # make variable
    im_dataC = Variable(im_dataC, volatile=True)
    im_infoC = Variable(im_infoC, volatile=True)
    num_boxesC = Variable(num_boxesC, volatile=True)
    gt_boxesC = Variable(gt_boxesC, volatile=True)

    im_file = os.path.join(imagePath)
            # im = cv2.imread(im_file)
    im_in = np.array(imread(im_file))
    if len(im_in.shape) == 2:
        im_in = im_in[:,:,np.newaxis]
        im_in = np.concatenate((im_in,im_in,im_in), axis=2)
          # rgb -> bgr
    im = im_in[:,:,::-1]
    blobs, im_scales = _get_image_blob(im)
    assert len(im_scales) == 1, "Only single-image batch implemented"
    im_blob = blobs
    im_info_np = np.array([[im_blob.shape[1], im_blob.shape[2], im_scales[0]]], dtype=np.float32)

    im_data_pt = torch.from_numpy(im_blob)
    im_data_pt = im_data_pt.permute(0, 3, 1, 2)
    im_info_pt = torch.from_numpy(im_info_np)

    im_dataC.data.resize_(im_data_pt.size()).copy_(im_data_pt)
    im_infoC.data.resize_(im_info_pt.size()).copy_(im_info_pt)
    gt_boxesC.data.resize_(1, 1, 5).zero_()
    num_boxesC.data.resize_(1).zero_()

    # pdb.set_trace()
    det_tic = time.time()

    rois, cls_prob, bbox_pred, \
    rpn_loss_cls, rpn_loss_box, \
    RCNN_loss_cls, RCNN_loss_bbox, \
    rois_label = model(im_dataC, im_infoC, gt_boxesC, num_boxesC)

    scores = cls_prob.data
    boxes = rois.data[:, :, 1:5]

In [15]:
hooklist = []
for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e):
        hooklist.append(e.register_forward_hook(getFeaturesMap))

In [16]:
pruneText = "data/HF_combined/HF_combined/ImageSets/Main/prune.txt"
imgFolder = "data/HF_combined/HF_combined/JPEGImages/"
img_extension = ".jpg"
allFM = {}
numOfImages = 1000

for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e):
        allFM[e] = []

with open(pruneText, 'r') as f:
    i = 0
    for imgName in f:
        i += 1
        imgPath = os.path.join(imgFolder, imgName.rstrip() + img_extension)
        run1Forward(fasterRCNN , imgPath)


/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


In [17]:
def getSortedIndex(convTensor, allFM):
    Hi_0 = []
    for i in range(allFM[convTensor].shape[1]):
        hist, bin_edges = np.histogram(allFM[convTensor][i,:], bins=10)
        p = hist / allFM[convTensor].shape[0]
        p = p[np.nonzero(p)]
        Hi_0.append((p * np.log(p)).sum())

    Hi_0 = np.asarray(Hi_0)
    d = np.zeros([2, Hi_0.shape[0]])
    i = np.argsort(Hi_0)
    return i.astype(int)

In [18]:
prevOutput = 3
prevTensor = None
compressionRate = 70
for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e) and i != 28:
        if prevOutput != e.in_channels:
            e = pruneNextLayer(e, prevOutput, rindexes)
        index = getSortedIndex(e, allFM)
        num2Removes = int(index.shape[0] * compressionRate / 100.)
        _, rindexes = pruneConvWithIndexes(e, index[num2Removes:])
        pruneCBias(e, rindexes)
        prevOutput = e.out_channels
        prevTensor = e
    if i == 28:
        e = pruneNextLayer(e, prevOutput, rindexes)
        continue

In [19]:
for i in hooklist:
    i.remove()

In [20]:
fasterRCNN.RCNN_base

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (17): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), p

In [238]:
pickle.dump(allFM, open("/tmp/allFM.p", "wb" ))

In [253]:
evalNetwork(fasterRCNN)

Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.071s 0.001s   
Writing head VOC results file


0.003003609493902607

In [ ]:
save_name_model = os.path.join("./pth_train_dir/vgg16/finished_models/", 'pruned_50_entropy.p')
torch.save(fasterRCNN, save_name_model)

In [35]:
save_name_model = os.path.join("./pth_train_dir/vgg16/finished_models/", 'pruned_50_entropy.p')
fasterRCNN = torch.load(save_name_model)

In [50]:
checkpoint = torch.load("pth_train_dir/vgg16/pruned_entropy_50/vgg16/holly/faster_rcnn_1_1_170000.pth")
fasterRCNN.load_state_dict(checkpoint['model'])

In [51]:
evalNetwork(fasterRCNN)

Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.038s 0.001s   
Writing head VOC results file


0.7249153714969145

In [14]:
allFM =  pickle.load(open("/tmp/allFM.p", "rb" ))

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/allFM.p'